In [1]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 

spark.sparkContext.uiWebUrl

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@40c28d06


Some(http://iuni2:4040)

In [3]:
//val sqlContext = new SQLContext(sc)
val df = sqlContext.read
  .format("com.databricks.spark.xml")
  .option("rowTag", "REC")
  //.schema(customSchema)
  .load("/WoSraw/WoS2019/xmlRaw/*.xml.gz")

df.show(2)
df.printSchema()

+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+----+--------+---------+-----------------+--------------------+----+
|                UID|    _r_id_disclaimer|abstracts|addresses|category_info|citedWork|contributors|        dynamic_data|fund_ack|   i|item|keywords|reference|reprint_addresses|         static_data| sub|
+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+----+--------+---------+-----------------+--------------------+----+
|WOS:000205001600028|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|null|    null|     null|             null|[,,,,, [, [, 1, [...|null|
|WOS:000202412000001|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|null|    null|     null|             null|[,,,,, [, [,

df = [UID: string, _r_id_disclaimer: string ... 14 more fields]


[UID: string, _r_id_disclaimer: string ... 14 more fields]

In [4]:
import org.apache.spark.sql.SaveMode
df.write.mode(SaveMode.Overwrite)
.parquet("/WoSraw/WoS2019/parquet")
//WoS2.describe().filter($"summary" === "count").show

In [23]:
val WoS = spark.read.format("parquet").load("/WoSraw/WoS2019/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name", "static_data.fullrecord_metadata.reprint_addresses",
                      "static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.category_info","static_data.item.keywords_plus",
                      "static_data.fullrecord_metadata.keywords.keyword","static_data.fullrecord_metadata.references.reference",
                      "static_data.contributors.contributor","static_data.summary.EWUID",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _r_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 

WoS = [UID: string, _r_id_disclaimer: string ... 14 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 16 more fields]


lastException: Throwable = null


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 16 more fields]

In [40]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

//WoS2.write.mode(SaveMode.Overwrite).json("/WoSraw/WoS2019/json")

//WoS2.write.mode(SaveMode.Overwrite).parquet("s3a://wos-athena/wos2019raw")

val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").orderBy($"citing".desc).write.mode(SaveMode.Overwrite)
    .parquet("s3a://wos-athena/wos2019raw/refEdges/")

//val test = spark.read.format("csv").load("s3a://wos-athena/NeuroJournalsAzure.tsv")
//test.show

WoSref = [citing: string, refList: array<string> ... 1 more field]


[citing: string, refList: array<string> ... 1 more field]

In [3]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)
val sdf = session.read.csv(
    "wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/metaInstitutes-08-03.csv"
)
sdf.show()

+----+--------+--------------------+--------+--------------------+--------------------+---------+----------+---------+--------------------+------------+--------------------+--------------------+------+----------------+--------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
| _c0|     _c1|                 _c2|     _c3|                 _c4|                 _c5|      _c6|       _c7|      _c8|                 _c9|        _c10|                _c11|                _c12|  _c13|            _c14|    _c15|                _c16|                _c17|                _c18| _c19|                _c20|                _c21|
+----+--------+--------------------+--------+--------------------+--------------------+---------+----------+---------+--------------------+------------+--------------------+--------------------+------+----------------+--------+--------------------+--------------------+--------------------+-----+--------------------+-----

session = org.apache.spark.sql.SparkSession@6c73d6d5
azureFile = /azureBlob.txt MapPartitionsRDD[6] at textFile at <console>:45
sdf = [_c0: string, _c1: string ... 20 more fields]


[_c0: string, _c1: string ... 20 more fields]

In [49]:
sdf.write.mode(SaveMode.Overwrite)
    .csv("wasbs://mag-2020-02-14@iunimag.blob.core.windows.net/processed_data.csv")

lastException: Throwable = null


In [5]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/WoSraw/WoS2019/json/refEdges.csv")

Name: org.apache.spark.SparkException
Message: Job aborted.
StackTrace:   at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.

In [7]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").orderBy($"cited".desc).coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/WoSraw/WoS2019/json/citeEdges.csv/")

WoSref = [citing: string, refList: array<string> ... 1 more field]


[citing: string, refList: array<string> ... 1 more field]

In [24]:
val WoS3 = WoS2.select("UID", "pub_info._pubtype", "pub_info._pubyear", "fund_ack.fund_text", "titles", "conference", "keyword", "keywords_plus", "abstract_text",
                       "identifier", "name","publishers.publisher.names.name.full_name","address_name.address_spec.full_address","category_info")
val WoS4 = WoS3.withColumn("allAddresses", array_join($"full_address", "|")).cache()
WoS4.describe().filter($"summary" === "count").show

+-------+--------+--------+--------+---------+------------+
|summary|     UID|_pubtype|_pubyear|full_name|allAddresses|
+-------+--------+--------+--------+---------+------------+
|  count|74883966|74883966|74883966| 74588122|    52459808|
+-------+--------+--------+--------+---------+------------+



WoS3 = [UID: string, _pubtype: string ... 12 more fields]
WoS4 = [UID: string, _pubtype: string ... 13 more fields]


[UID: string, _pubtype: string ... 13 more fields]

In [20]:
WoS4.printSchema

root
 |-- UID: string (nullable = true)
 |-- _pubtype: string (nullable = true)
 |-- _pubyear: long (nullable = true)
 |-- fund_text: struct (nullable = true)
 |    |-- p: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- conference: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _conf_id: long (nullable = true)
 |    |    |-- conf_dates: struct (nullable = true)
 |    |    |    |-- _count: long (nullable = true)
 |    |    |    |-- conf_date: struct (nullable = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _conf_end: string (nullable = true)


In [25]:
//WoSIU2.describe().filter($"summary" === "count").show
val WoS5 = WoS4.withColumn("FX", array_join($"fund_text.p", ".")).withColumn("AB", array_join($"abstract_text.p", "."))
WoS5.show

+-------------------+--------+--------+---------+--------------------+----------+-------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+
|                UID|_pubtype|_pubyear|fund_text|              titles|conference|keyword|keywords_plus|abstract_text|          identifier|                name|           full_name|        full_address|       category_info|        allAddresses|  FX|  AB|
+-------------------+--------+--------+---------+--------------------+----------+-------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+
|WOS:A1985AUK0600019| Journal|    1985|     null|[6, [[CLINICA CHI...|      null|   null|         null|         null|[[, accession_no,...|[[,,, Y, author, ...| ELSEVIER SCIENCE BV|                null|[[1, [Science & T...|                

WoS5 = [UID: string, _pubtype: string ... 15 more fields]


[UID: string, _pubtype: string ... 15 more fields]

In [26]:
import scala.util.Try
import org.apache.hadoop.fs._
import org.apache.spark.sql.Row

def getTitle(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, _, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}
val WoS6 = WoS5.withColumn("journal", when(col("titles.title").isNotNull, getTitle("source")($"titles.title")).otherwise(lit(null))).cache()
WoS6.show()

+-------------------+--------+--------+---------+--------------------+----------+-------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+--------------------+
|                UID|_pubtype|_pubyear|fund_text|              titles|conference|keyword|keywords_plus|abstract_text|          identifier|                name|           full_name|        full_address|       category_info|        allAddresses|  FX|  AB|             journal|
+-------------------+--------+--------+---------+--------------------+----------+-------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+--------------------+
|WOS:A1985AUK0600019| Journal|    1985|     null|[6, [[CLINICA CHI...|      null|   null|         null|         null|[[, accession_no,...|[[,,, Y, author, ...| ELSEVIER SCIENC

WoS6 = [UID: string, _pubtype: string ... 16 more fields]


getTitle: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, _pubtype: string ... 16 more fields]

In [27]:
//WoSIU4.write.mode(SaveMode.Overwrite).parquet("/WoSraw/WoS2019/IUpapersExtract")
//val WoSIU4b = spark.read.format("parquet").load("/WoSraw/WoS2019/IUpapersExtract/*.parquet")
val WoS7 = WoS6.withColumn("title", when(col("titles.title").isNotNull, getTitle("item")($"titles.title")).otherwise(lit(null)))
WoS7.show

+-------------------+--------+--------+---------+--------------------+----------+-------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+--------------------+--------------------+
|                UID|_pubtype|_pubyear|fund_text|              titles|conference|keyword|keywords_plus|abstract_text|          identifier|                name|           full_name|        full_address|       category_info|        allAddresses|  FX|  AB|             journal|               title|
+-------------------+--------+--------+---------+--------------------+----------+-------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+--------------------+--------------------+
|WOS:A1985AUK0600019| Journal|    1985|     null|[6, [[CLINICA CHI...|      null|   null|         null|         

WoS7 = [UID: string, _pubtype: string ... 17 more fields]


[UID: string, _pubtype: string ... 17 more fields]

In [28]:
def getID(_type: String) = {
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, s, _) => s == _type}
      .map(_._3)
      .orElse(None)
  )
}


val WoS8 = WoS7.withColumn("doi", when(col("identifier").isNotNull, getID("doi")($"identifier")).otherwise(lit(null)))
.withColumn("issn", when(col("identifier").isNotNull, getID("issn")($"identifier")).otherwise(lit(null)))
.withColumn("eissn", when(col("identifier").isNotNull, getID("eissn")($"identifier")).otherwise(lit(null))).cache()
//.select("UID","doi","issn","eissn").filter($"doi" =!= "null").show
WoS8.describe().filter($"summary" === "count").show

+-------+--------+--------+--------+---------+------------+--------+--------+--------+--------+--------+--------+--------+
|summary|     UID|_pubtype|_pubyear|full_name|allAddresses|      FX|      AB| journal|   title|     doi|    issn|   eissn|
+-------+--------+--------+--------+---------+------------+--------+--------+--------+--------+--------+--------+--------+
|  count|74883966|74883966|74883966| 74588122|    52459808|11859531|35990463|74883485|74883956|24368416|68621122|21048392|
+-------+--------+--------+--------+---------+------------+--------+--------+--------+--------+--------+--------+--------+



WoS8 = [UID: string, _pubtype: string ... 20 more fields]


getID: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, _pubtype: string ... 20 more fields]

In [29]:
WoS8.printSchema

root
 |-- UID: string (nullable = true)
 |-- _pubtype: string (nullable = true)
 |-- _pubyear: long (nullable = true)
 |-- fund_text: struct (nullable = true)
 |    |-- p: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- conference: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _conf_id: long (nullable = true)
 |    |    |-- conf_dates: struct (nullable = true)
 |    |    |    |-- _count: long (nullable = true)
 |    |    |    |-- conf_date: struct (nullable = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _conf_end: string (nullable = true)


In [30]:
val WoS9 = WoS8.withColumn("reprintFlag", array_position($"name._reprint", "Y"))
            .select("UID","_pubtype","_pubyear","eissn","issn","doi","title","AB","journal","full_name","FX","reprintFlag","name.display_name","allAddresses",
                    "conference.conf_titles.conf_title","keyword","keywords_plus","category_info.headings.heading","category_info.subheadings.subheading","category_info.subjects.subject")
            .toDF("WoSid","PT","PY","EI","SN","DI","TI","AB","SO","PU","FX","RP","authors","C1","conference","keyword","keywords_plus","heading","subheading","subject").withColumn("AU", array_join($"authors", "|"))
WoS9.show(false)

+-------------------+-------+----+---------+---------+----+----------------------------------------------------------------------------------------------------------------------------------------+----+----------------------------------------------------------------------------------------------------+----------------------------+----+---+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------+----------+-------+-------------+---------------------------------------+-------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------+
|WoSid              |PT     |PY  |EI       |SN       |DI  |TI          

WoS9 = [WoSid: string, PT: string ... 19 more fields]


[WoSid: string, PT: string ... 19 more fields]

In [37]:
//"conference", "keyword", "abstract_text",
val WoS10 = WoS9.withColumn("DE", array_join($"keyword", ";")).withColumn("CT", array_join($"conference", ";"))
WoS10.show
//WoS10.describe().filter($"summary" === "count").show

+-------------------+-------+----+---------+---------+----+--------------------+----+--------------------+--------------------+----+---+--------------------+--------------------+----------+-------+-------------+--------------------+--------------------+--------------------+--------------------+----+----+
|              WoSid|     PT|  PY|       EI|       SN|  DI|                  TI|  AB|                  SO|                  PU|  FX| RP|             authors|                  C1|conference|keyword|keywords_plus|             heading|          subheading|             subject|                  AU|  DE|  CT|
+-------------------+-------+----+---------+---------+----+--------------------+----+--------------------+--------------------+----+---+--------------------+--------------------+----------+-------+-------------+--------------------+--------------------+--------------------+--------------------+----+----+
|WOS:A1985AUK0600019|Journal|1985|     null|0009-8981|null|SERUM FERRITIN IN...|nu

WoS10 = [WoSid: string, PT: string ... 21 more fields]


[WoSid: string, PT: string ... 21 more fields]

In [38]:
val WoS11 = WoS10.withColumn("ID", array_join($"keywords_plus.keyword", ";"))
WoS11.select("ID").filter($"ID" =!= "null").show(false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID                                                                                                                                                                         |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|EGYPT;DISEASE;CATTLE;SHEEP                                                                                                                                                 |
|CHARGE-TRANSFER PROCESSES                                                                                                                                                  |
|OXYGEN-ISOTOPE RATIOS;IGNEOUS ROCKS;HYDROGEN;COMPLEX;QUARTZ;CHERT;MULL;SKYE                                                      

WoS11 = [WoSid: string, PT: string ... 22 more fields]


[WoSid: string, PT: string ... 22 more fields]

In [39]:
def getSubject(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1)))
      .find { case (_, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}

val WoS12 = WoS11.withColumn("headings", array_join($"heading", ","))
                .withColumn("subheadings", array_join($"subheading", ","))
                //.withColumn("WC/SC", array_join($"subject", ";"))
                .withColumn("WC", when(col("subject").isNotNull, getSubject("traditional")($"subject")).otherwise(lit(null)))
                .withColumn("SC", when(col("subject").isNotNull, getSubject("extended")($"subject")).otherwise(lit(null)))
WoS12.show(false)

+-------------------+-------+----+---------+---------+----+----------------------------------------------------------------------------------------------------------------------------------------+----+----------------------------------------------------------------------------------------------------+----------------------------+----+---+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------+----------+-------+-------------+---------------------------------------+-------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------+----+----+----+------------------------------------+--------------------

WoS12 = [WoSid: string, PT: string ... 26 more fields]


getSubject: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[WoSid: string, PT: string ... 26 more fields]

In [38]:
WoS12.select("set","WoSid","PT","PY","EI","SN","DI","TI","AB","DE","ID","CT","SO","PU","FX","RP","AU","C1"
             ,"headings","subheadings","WC","SC")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/data1/nobel_papers.csv")

+-------------------+----+----+----+----+----+------------------------------------------------------------------------------------------+----+-----------------------------------------------------------------------------------------------+---------------------------------+----+---+-------------------------------------------------------------------------+----+----------------------------------------------------------------------------+-------+-------------------------------------------------------------------+----+--------------------------------------------------------------------------+
|WoSid              |PT  |PY  |EI  |SN  |DI  |TI                                                                                        |AB  |SO                                                                                             |PU                               |FX  |RP |authors                                                                  |C1  |conference                                    

In [41]:
WoS12.select("WoSid","DE","ID","headings","subheadings","WC","SC")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/WoSraw/WoS2019/category_keywords.csv")

lastException: Throwable = null
